In [ ]:
from pyspark import SparkContext, SparkConf

# Initialize Spark context
conf = SparkConf().setAppName("GPA Calculation and Sorting")
sc = SparkContext.getOrCreate(conf=conf)

# Read the input file
data = sc.textFile("inpufile.txt")  

# Function to convert grades to GPA
def grade_to_gpa(grade):
    if grade == 'A':
        return 4
    elif grade == 'B':
        return 3
    elif grade == 'C':
        return 2
    elif grade == 'D':
        return 1
    else:
        return 0

# Split data and calculate GPA
subject_gpa = data.map(lambda line: line.split(' ')).map(lambda x: ((x[0], x[2]), grade_to_gpa(x[3])))

# Calculate average GPA for each subject
subject_avg_gpa = subject_gpa.mapValues(lambda x: (x, 1)).reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
    .mapValues(lambda x: x[0] / x[1])



In [ ]:
# Function to sort roll numbers based on specified rules
def custom_sort(roll):
    parts = roll.split('-')
    campus_order = {'F': 0, 'I': 1, 'K': 2, 'L': 3, 'P': 4}
    year_order = {'99': 0, '01': 1, '16': 2, '17': 3}
    return (campus_order[parts[0][0]], year_order[parts[0][1:3]], int(parts[1]))

# Sort based on roll number
sorted_data = subject_avg_gpa.sortByKey(keyfunc=lambda x: custom_sort(x[0]))

In [ ]:
# Filter the data for the given roll number
specific_roll = sorted_data.filter(lambda x: x[0][0] == 'L21-1705')

# Collect and print the result
result = specific_roll.collect()
for item in result:
    print(f"Roll Number: {item[0]}, Average GPA: {item[1]}")

sc.stop()